In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .appName('resp-entregavel_3')\
                    .getOrCreate()

22/12/09 09:43:14 WARN Utils: Your hostname, BRSPOBITANL1859 resolves to a loopback address: 127.0.1.1; using 192.168.0.60 instead (on interface wlp0s20f3)
22/12/09 09:43:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 09:43:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/09 09:43:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Para possibilitar a leitura de um 'nested json', dessa vez em uma das opções de leitura do arquivo, vamos utilizar o "multiline" e "true".

In [3]:
df=spark.read.option("multiline","true").json("data.json")
df.show()


+--------------------+--------+-------------------+--------------------+-----+---------+
|          CreateDate|Discount|       EmissionDate|            ItemList|NFeID|NFeNumber|
+--------------------+--------+-------------------+--------------------+-----+---------+
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|[{Rice, 2, 35.55}...|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|[{Tomate, 10, 12....|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|[{Beer, 6, 9.0}, ...|    3|      503|
+--------------------+--------+-------------------+--------------------+-----+---------+



Utilizando a função explode do pyspark.sql para 'explodir' o array dentro da coluna ItemList em linhas

In [4]:
from pyspark.sql import functions as f

df = df.withColumn('ItemList', f.explode('ItemList')).select('ItemList.*', '*').drop('ItemList')
df.show()


+------------+--------+-----+--------------------+--------+-------------------+-----+---------+
| ProductName|Quantity|Value|          CreateDate|Discount|       EmissionDate|NFeID|NFeNumber|
+------------+--------+-----+--------------------+--------+-------------------+-----+---------+
|        Rice|       2|35.55|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|       Flour|       5|11.55|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|        Bean|       7|27.15|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|      Tomate|      10|12.25|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|       Pasta|       5| 7.55|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|        Beer|       6|  9.0|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|French fries|       2|10.99|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|   Ice cream|       1|27.15|2021-05-24T

Pronto! Agora já temos a primeira parte da atividade 'Expandir a coluna num mesmo dataframe' feita. No próximo passo precisamos normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.

Para o próximo passo vamos criar 2 tabelas (dataframes), um chamado para o itemlist e outro para a NFe.

In [5]:
# Selecionando as colunas que compõem a tabela itemlist e adicionando NFeID como chave estrangeira (FK)

df_itemlist = df.select('ProductName', 'Quantity', 'Value', 'NFeID')
df_itemlist.show()

+------------+--------+-----+-----+
| ProductName|Quantity|Value|NFeID|
+------------+--------+-----+-----+
|        Rice|       2|35.55|    1|
|       Flour|       5|11.55|    1|
|        Bean|       7|27.15|    1|
|      Tomate|      10|12.25|    2|
|       Pasta|       5| 7.55|    2|
|        Beer|       6|  9.0|    3|
|French fries|       2|10.99|    3|
|   Ice cream|       1|27.15|    3|
+------------+--------+-----+-----+



In [6]:
# Dropando as colunas correspondentes a itemlist

df_nfe = df.drop('ProductName', 'Quantity', 'Value')
df_nfe.show()

+--------------------+--------+-------------------+-----+---------+
|          CreateDate|Discount|       EmissionDate|NFeID|NFeNumber|
+--------------------+--------+-------------------+-----+---------+
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
+--------------------+--------+-------------------+-----+---------+



Como podemos ver no dataframe gerado, as linhas estão repetidas. Para resolver isso vamos usar a função distinct do pyspark.

In [7]:
df_nfe.distinct().show()

+--------------------+--------+-------------------+-----+---------+
|          CreateDate|Discount|       EmissionDate|NFeID|NFeNumber|
+--------------------+--------+-------------------+-----+---------+
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
+--------------------+--------+-------------------+-----+---------+



Pronto, normalizamos a tabela e dividimos em dois dataframes separados, seguindo o modelo relacional. Vamos dar uma olhada nelas de novo.

## NFe

In [8]:
df_nfe.show()

+--------------------+--------+-------------------+-----+---------+
|          CreateDate|Discount|       EmissionDate|NFeID|NFeNumber|
+--------------------+--------+-------------------+-----+---------+
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    1|      501|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    2|      502|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
|2021-05-24T20:21:...|     0.0|2021-05-24T00:00:00|    3|      503|
+--------------------+--------+-------------------+-----+---------+



## ItemList

In [9]:
df_itemlist.show()

+------------+--------+-----+-----+
| ProductName|Quantity|Value|NFeID|
+------------+--------+-----+-----+
|        Rice|       2|35.55|    1|
|       Flour|       5|11.55|    1|
|        Bean|       7|27.15|    1|
|      Tomate|      10|12.25|    2|
|       Pasta|       5| 7.55|    2|
|        Beer|       6|  9.0|    3|
|French fries|       2|10.99|    3|
|   Ice cream|       1|27.15|    3|
+------------+--------+-----+-----+



# Diagrama ER

![Diagrama](src/diagrama.png)